In [ ]:
# Import the necessary modules for creating the dashboard and interacting with MongoDB
from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import dash_leaflet as dl
import pandas as pd
import plotly.express as px  # Import for creating the pie chart
from animal_shelter import AnimalShelter  # Custom module to interact with MongoDB

# MongoDB credentials and connection details
username = "aacuser"  # MongoDB username
password = "SNHU1234"  # MongoDB password
host = "nv-desktop-services.apporto.com"  # Host where MongoDB is running 
port = "30415"  # Default MongoDB port
db = "AAC"  # The name of the database to connect to
collection = "animals"  # The name of the collection within the database

# Initialize the AnimalShelter class, connecting to MongoDB using the provided credentials
shelter = AnimalShelter(username=username, password=password, host=host, port=port, db=db, collection=collection)

# Fetch all data from MongoDB and store it as a Pandas DataFrame
df = pd.DataFrame.from_records(shelter.read({}))

# Check if the DataFrame contains data; if so, drop the MongoDB '_id' field, which is not needed for the display
if not df.empty:
    df.drop(columns=['_id'], inplace=True)
    print(f"Total records fetched from MongoDB: {len(df)}")  # Debugging statement to verify data fetch
else:
    print("No data was fetched from MongoDB.")

# Function to filter out breeds that make up less than 1% of the total, to avoid displaying less significant data
def filter_breeds_over_one_percent(dataframe):
    breed_counts = dataframe['breed'].value_counts(normalize=True) * 100  # Calculate the percentage of each breed
    large_breeds = breed_counts[breed_counts >= 1].index  # Filter breeds that account for at least 1% of the total
    filtered_df = dataframe[dataframe['breed'].isin(large_breeds)]  # Return only rows where the breed meets the criteria
    return filtered_df

# Initialize the Dash app
app = Dash(__name__)

# Define the layout for the dashboard
app.layout = html.Div([
    # Top section with the logo and title side by side
    html.Div([
        # Add a clickable logo image linking to the SNHU website, which opens in a new window
        html.A(
            href='https://www.snhu.edu',
            target="_blank",  # Open link in a new tab or window
            children=[
                html.Img(src='/assets/Grazioso Salvare Logo.png', style={'height': '80px', 'margin-right': '20px'})
            ]
        ),
        # Center the title next to the logo
        html.Center(html.B(html.H1('Austin Animal Center Outcomes')))
    ], style={'display': 'flex', 'align-items': 'center'}),  # Align the logo and title horizontally

    # Horizontal line separator for visual separation
    html.Hr(),

    # Radio buttons to filter by rescue type, with 'Reset' pre-selected to show all data initially
    dcc.RadioItems(
        id='rescue-type-radio',
        options=[
            {'label': 'Water Rescue', 'value': 'Water Rescue'},
            {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
            {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
            {'label': 'Reset (Unfiltered)', 'value': 'Reset'}
        ],
        value='Reset',  # Set 'Reset' as the default selection
        labelStyle={'display': 'block'},  # Display the radio buttons in a vertical list
        inputStyle={'margin-right': '10px'}  # Add space between the radio button and label
    ),

    # Space between the radio buttons and the data table
    html.Br(),

    # Data table to display the MongoDB data, with sorting enabled and a limit of 10 rows per page
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=[],  # Initialize with empty data; callback will populate
        row_selectable="single",  # Allow single row selection for further interaction
        selected_rows=[],  # No rows selected initially
        page_size=10,  # Display only 10 rows at a time
        sort_action='native',  # Enable native sorting by clicking on column headers
        style_table={'height': '300px', 'overflowY': 'auto'},  # Make the table scrollable
        style_cell={
            'textAlign': 'left',
            'padding': '5px'
        },
        style_header={
            'backgroundColor': 'rgb(230, 230, 230)',
            'fontWeight': 'bold'
        }
    ),

    # Space between the data table and the charts
    html.Br(),

    # Container for pie chart and map side by side
    html.Div([
        # Pie chart to display the breed distribution based on the **filtered data**
        dcc.Graph(
            id='pie-chart',
            figure={},  # Initialize with empty figure; callback will populate
            style={'flex': '1', 'padding': '10px'}
        ),

        # Placeholder for the interactive map to display the location of selected animals
        html.Div(
            id='map-id',
            className='col s12 m6',
            style={'flex': '1', 'padding': '10px'}
        )
    ], style={'display': 'flex', 'flexWrap': 'wrap'}),  # Use flexbox to arrange children side by side

    # Footer for identification purposes
    html.Div(
        "Tom Strogen SNHU CS-340 MongoDB Authentication",
        style={'marginTop': '20px', 'fontStyle': 'italic'}
    )
])

# Callback to highlight selected columns in the data table
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'  # Apply a light blue background to the selected columns
    } for i in selected_columns]

# Callback to filter the data table and update the pie chart based on the selected radio button
@app.callback(
    [Output('datatable-id', 'data'), Output('pie-chart', 'figure')],
    [Input('rescue-type-radio', 'value')]
)
def update_table_and_pie_chart(rescue_type):
    # Initialize an empty DataFrame
    filtered_df = pd.DataFrame()

    # Mapping from rescue_type to filter_type
    rescue_to_filter = {
        'Water Rescue': 'Water',
        'Mountain or Wilderness Rescue': 'Mountain',
        'Disaster or Individual Tracking': 'Disaster'
    }

    filter_type = rescue_to_filter.get(rescue_type, 'Reset')  # Default to 'Reset' if not found

    if filter_type == 'Water':
        query = {
            "$and": [
                {
                    "$or": [
                        {"breed": {"$regex": 'Labrador Retriever Mix', "$options": "i"}},
                        {"breed": {"$regex": 'Chesapeake', "$options": "i"}},
                        {"breed": {"$regex": 'Newfoundland', "$options": "i"}}
                    ]
                },
                {"sex_upon_outcome": 'Intact Female'},
                {
                    "age_upon_outcome_in_weeks": {
                        "$gte": 26,
                        "$lte": 156
                    }
                }
            ]
        }
        filtered_df = pd.DataFrame.from_records(shelter.read(query))

    elif filter_type == 'Mountain':
        query = {
            "$and": [
                {
                    "$or": [
                        {"breed": {"$regex": 'German Shepherd', "$options": "i"}},
                        {"breed": {"$regex": 'Alaskan Malamute', "$options": "i"}},
                        {"breed": {"$regex": 'Old English Sheepdog', "$options": "i"}},
                        {"breed": {"$regex": 'Siberian Husky', "$options": "i"}},
                        {"breed": {"$regex": 'Rottweiler', "$options": "i"}}
                    ]
                },
                {"sex_upon_outcome": 'Intact Male'},
                {
                    "age_upon_outcome_in_weeks": {
                        "$gte": 26,
                        "$lte": 156
                    }
                }
            ]
        }
        filtered_df = pd.DataFrame.from_records(shelter.read(query))

    elif filter_type == 'Disaster':
        query = {
            "$and": [
                {
                    "$or": [
                        {"breed": {"$regex": 'Doberman Pinscher', "$options": "i"}},
                        {"breed": {"$regex": 'German Shepherd', "$options": "i"}}, 
                        {"breed": {"$regex": 'Golden Retriever', "$options": "i"}},  
                        {"breed": {"$regex": 'Bloodhound', "$options": "i"}},
                        {"breed": {"$regex": 'Rottweiler', "$options": "i"}}
                    ]
                },
                {"sex_upon_outcome": 'Intact Male'},
                {
                    "age_upon_outcome_in_weeks": {
                        "$gte": 20,
                        "$lte": 300
                    }
                }
            ]
        }
        filtered_df = pd.DataFrame.from_records(shelter.read(query))

    else:
        # Reset: Show all entries without additional filters
        filtered_df = pd.DataFrame.from_records(shelter.read({}))

    # Check if the filtered DataFrame contains data
    if not filtered_df.empty:
        # Drop the MongoDB '_id' field if present
        if '_id' in filtered_df.columns:
            filtered_df.drop(columns=['_id'], inplace=True)
    else:
        print("No data matches the selected rescue type.")

    # Update the pie chart based on the **filtered data**
    # Note: To maintain consistency, the pie chart should always display breeds ≥ 1% of the **current filtered data**
    pie_chart_filtered_df = filter_breeds_over_one_percent(filtered_df)
    pie_chart_figure = px.pie(
        pie_chart_filtered_df,
        names='breed',
        title="Breed Distribution (≥ 1%)",
        hole=0.3  # Optional: Adds a donut hole for aesthetics
    )

    return filtered_df.to_dict('records'), pie_chart_figure

# Callback to update the map based on the selected row in the data table
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")]
)
def update_map(viewData, selected_rows):
    # Check if there is data to display
    if viewData is None or len(viewData) == 0:
        return [html.P("No data available to display on the map.")]

    # Convert the virtual data to a DataFrame
    dff = pd.DataFrame.from_dict(viewData)

    # Determine which row is selected; default to the first row if none are selected
    if selected_rows is None or len(selected_rows) == 0:
        row = 0  # Default to the first row
    else:
        row = selected_rows[0]

    # Ensure that the selected row exists and has the necessary location data
    if 'location_lat' in dff.columns and 'location_long' in dff.columns and dff.shape[0] > row:
        latitude = dff.loc[row, 'location_lat']
        longitude = dff.loc[row, 'location_long']

        # Check if latitude and longitude are valid numbers
        if pd.notnull(latitude) and pd.notnull(longitude):
            # Return a map centered around the selected animal's location, using its latitude and longitude
            return [
                dl.Map(
                    style={'width': '100%', 'height': '500px'},
                    center=[latitude, longitude],  # Center map on the animal's location
                    zoom=12,  # Adjust zoom level as needed
                    children=[
                        dl.TileLayer(id="base-layer-id"),  # Add base map layer
                        dl.Marker(
                            position=[latitude, longitude],  # Marker at the animal's location
                            children=[
                                dl.Tooltip(dff.iloc[row]['breed'] if 'breed' in dff.columns else "Breed Unknown"),  # Show breed as a tooltip
                                dl.Popup([
                                    html.H3("Animal Details"),
                                    html.P(f"Name: {dff.iloc[row]['animal_name']}" if 'animal_name' in dff.columns else "Name Unknown"),
                                    html.P(f"Age (weeks): {dff.iloc[row]['age_upon_outcome_in_weeks']}" if 'age_upon_outcome_in_weeks' in dff.columns else "Age Unknown"),
                                    html.P(f"Rescue Type: {dff.iloc[row]['rescue_type']}" if 'rescue_type' in dff.columns else "Rescue Type Unknown")
                                ])
                            ]
                        )
                    ]
                )
            ]
        else:
            return [html.P("Selected row does not have valid location data.")]
    else:
        return [html.P("Selected row does not have location data.")]

# Run the Dash application with debug mode enabled
if __name__ == '__main__':
    app.run_server(debug=True, use_reloader=False)


Successfully connected to MongoDB!
Found 10004 document(s) matching the query.
Total records fetched from MongoDB: 10004
Dash is running on http://127.0.0.1:25900/

 * Serving Flask app '__main__'
 * Debug mode: on
Found 10004 document(s) matching the query.
Found 19 document(s) matching the query.
Found 28 document(s) matching the query.
Found 17 document(s) matching the query.
